# 딥러닝과 영상 인식  
  
컨볼루션 신경망(CNN: Convolutional Neural Network)  
• 영상 인식 등을 위한 딥러닝에 특화된 네트워크 구조  
• CNN의 목적은 첫 번째는 특성 추출이고 두 번째는 추출 된 모든 특성을 집계하고이를 기반으로 예측하는 것  
• 일반적 구성: 컨볼루션(convolution) + 풀링(pooling) + … + 완전 연결 레이어(FC)  
  
![14-1.jpg](attachment:14-1.jpg)  
  
![14_2.jpg](attachment:14_2.jpg)
  
컨볼루션 레이어(Convolution Layer)  
• 2차원 영상에서 유효한 특징(feature)를 찾아내는 역할  
• 연산의 주요 목적은 입력 이미지에서 가장자리, 곡선, 모서리, 그라데이션 방향 등과 같은 특징을 추출  
• 입력 이미지의 모든 수평 및 수직 가장자리를 추출  
• 유용한 필터 마스크가 학습에 의해 결정됨  
• 보통 ReLU 활성화 함수를 함께 사용함  
  
  ![14_3.jpg](attachment:14_3.jpg)  
  
입력 크기 : 6 * 6 * 3        
필터 크기 : 3 * 3 * 3(입력 깊이 = 3이므로 필터 깊이도 3)  
필터 개수 : 2  
한 필터의 가중치 = 3 * 3 * 3 = 27  
총 가중치 = 27 * 2 = 54  
  
가중치 값은 임의의 값으로 초기화되며 모든 학습 단계에서 계속 조정.  
  
출력 맵 차수계산  
입력(n * n), 필터(f, f)이면  
출력차수 = (n - j + 1)  
예> 입력(6 * 6), 필터(3, 3)이면 6-3+1=4 이므로  
출력차수 = 4 * 4  
  
패딩  
valid: 패딩이 추가되지 않아 출력 맵 축소됨.  
same: 패딩 추가. 출력 맵의 크기가 입력과 동일.  
  
stride  
이동거리: 보통 1. 1보다 크면 출력 맵 크기 축소됨. 정보누락. 메모리, 시간 절약됨  
  
출력 맵 차수 계산 공식  
입력(n), 필터(f), 패딩(p), stride(s)  
=> (n+2p-f/s+1) * (n+2p-f/s+1)  
  
입력(n) = (6x6), 필터(f) = (3x3), 패딩(p) = 1, stride(s) = 1 이면   
=>  
(6 + 2 * 1 – 3) / 1 + 1 = 6  
출력차수 = 6 x 6  
   
입력(n) = (7x7), 필터(f) = (3x3), 패딩(p) = 1, stride(s) = 2 이면   
=>   
(7 + 2 * 1-3) / 2 = 4  
출력차수 = 4 x 4  
  
  ![14_4.jpg](attachment:14_4.jpg)  
  
출력 맵의 16개의 모든 요소에 바이어스를 더해주고, 활성화 함수 ReLU 적용.  
비선형 모델로 변환하고 양수 결과는 같은 값을 반환하고 음수는 0을 반환  
첫 번째 컨볼루션 레이어는 가장자리, 색상, 그래디언트 방향 등과 같은 낮은 수준의 특징 캡처. 레이어를 추가하면 점점 섬세한 특징 추출  
  
풀링 레이어(Pooling Layer)  
• 유용한 정보는 유지하면서 입력 크기를 줄임으로써 과적합(overfitting)을 예방하고 계산량을 감소시키는 효과  
• 최대 풀링(max pooling) 또는 평균 풀링(average pooling) 사용  
• 학습이 필요 없음  
• 보통 2x2 창, 스트라이드 2를 사용
  
  ![14_5.jpg](attachment:14_5.jpg)  
  
완전 연결 레이어(Fully Connected Layer)  
• 3차원 구조의 activation map(H * W * C)의 모든 값을 일렬로 이어 붙임. flatten  
• 인식의 경우, 소프트맥스(softmax) 레이어를 추가하여 각 클래스에 대한 확률 값을 결과로 얻음  
  
  ![14_6.jpg](attachment:14_6.jpg)  
  
  ![14_7.jpg](attachment:14_7.jpg)

In [ ]:
import torch
import torch.nn as nn


<입력 정의>

In [ ]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


<합성곱 층>

In [ ]:
# input=1채널 / output=32채널 / 커널 사이즈는 3 / 패딩은 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# input=32채널 / output=64채널 / 커널 사이즈는 3 / 패딩은 1입
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# stride, kernel size 모두 2
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


< 모델 연결 >

In [ ]:
# 입력을 첫 콘브넷에 연결
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [ ]:
# 합성곱의 결과에 풀링
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [ ]:
# 풀링 결과를 다시 두번째 콘브넷에 연결
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [ ]:
# 두 번째 합성곱의 결과에 풀링
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


# CNN으로 MNIST 분류하기 

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

<기기 선택>

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

< learning rate, 반복회수, 미니 배치 크기 변수 정의> 

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

< 자료 다운 로드 >

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



< 데이터 로더로 미니 배치 생성 >

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

< CNN 클래스 정의 >

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    # 모든 모듈에는 forward(), backward() 함수를 갖는다 재정의
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

< 모델 생성 >

In [ ]:
model = CNN().to(device)

< 비용함수, 옵티마이저 생성 >

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

< 배치 크기 확인 >

In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


< 학습 >

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X) # forward() 실행
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225658983
[Epoch:    2] cost = 0.0629897192
[Epoch:    3] cost = 0.046263855
[Epoch:    4] cost = 0.0374744497
[Epoch:    5] cost = 0.0315009803
[Epoch:    6] cost = 0.0260562431
[Epoch:    7] cost = 0.0217889287
[Epoch:    8] cost = 0.0184159651
[Epoch:    9] cost = 0.0162835792
[Epoch:   10] cost = 0.0133258477
[Epoch:   11] cost = 0.0100290906
[Epoch:   12] cost = 0.00988505036
[Epoch:   13] cost = 0.00857405178
[Epoch:   14] cost = 0.00615035789
[Epoch:   15] cost = 0.00720831985


< 테스트 >

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9858999848365784


# 깊은 CNN으로 MNIST 분류하기

1. 모델 이해하기  
  
1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

3번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 64, out_channel = 128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2, padding=1))

4번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 4 × 4 × 128 → batch_size × 2048
전결합층(뉴런 625개) + 활성화 함수 ReLU

5번 레이어 : 전결합층(Fully-Connected layer)
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.191919655
[Epoch:    2] cost = 0.0543407872
[Epoch:    3] cost = 0.0370745286
[Epoch:    4] cost = 0.0304617751
[Epoch:    5] cost = 0.0256527681
[Epoch:    6] cost = 0.0207393486
[Epoch:    7] cost = 0.0157388318
[Epoch:    8] cost = 0.0154955527
[Epoch:    9] cost = 0.0134736961
[Epoch:   10] cost = 0.0122741275
[Epoch:   11] cost = 0.0108220233
[Epoch:   12] cost = 0.0104403114
[Epoch:   13] cost = 0.00696945796
[Epoch:   14] cost = 0.00789370202
[Epoch:   15] cost = 0.00861239526


In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9896000027656555
